In [1]:
# Built-in libaries
import argparse
#import collections
import datetime
import multiprocessing
import os
import time
# External libraries
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import xarray as xr
# Local libraries
import globaldebris_input as input

In [2]:
def split_list(lst, n=1, option_ordered=1):
    """
    Split list into batches for the supercomputer.
    
    Parameters
    ----------
    lst : list
        List that you want to split into separate batches
    n : int
        Number of batches to split glaciers into.
    
    Returns
    -------
    lst_batches : list
        list of n lists that have sequential values in each list
    """
    # If batches is more than list, then there will be one glacier in each batch
    if option_ordered == 1:
        if n > len(lst):
            n = len(lst)
        n_perlist_low = int(len(lst)/n)
        n_perlist_high = int(np.ceil(len(lst)/n))
        lst_copy = lst.copy()
        count = 0
        lst_batches = []
        for x in np.arange(n):
            count += 1
            if count <= len(lst) % n:
                lst_subset = lst_copy[0:n_perlist_high]
                lst_batches.append(lst_subset)
                [lst_copy.remove(i) for i in lst_subset]
            else:
                lst_subset = lst_copy[0:n_perlist_low]
                lst_batches.append(lst_subset)
                [lst_copy.remove(i) for i in lst_subset]
    
    else:
        if n > len(lst):
            n = len(lst)
    
        lst_batches = [[] for x in np.arange(n)]
        nbatch = 0
        for count, x in enumerate(lst):
            if count%n == 0:
                nbatch = 0
    
            lst_batches[nbatch].append(x)
            
            nbatch += 1
            
    return lst_batches    


def getparser():
    """
    Use argparse to add arguments from the command line

    Parameters
    ----------
    batchno (optional) : int
        batch number used to differentiate output on supercomputer
    batches (optional) : int
        total number of batches based on supercomputer
    num_simultaneous_processes (optional) : int
        number of cores to use in parallels
    option_parallels (optional) : int
        switch to use parallels or not
    debug (optional) : int
        Switch for turning debug printing on or off (default = 0 (off))

    Returns
    -------
    Object containing arguments and their respective values.
    """
    parser = argparse.ArgumentParser(description="run simulations from gcm list in parallel")
    # add arguments
    parser.add_argument('-batchno', action='store', type=int, default=0,
                        help='Batch number used to differentiate output on supercomputer')
    parser.add_argument('-batches', action='store', type=int, default=1,
                        help='Total number of batches (nodes) for supercomputer')
    parser.add_argument('-num_simultaneous_processes', action='store', type=int, default=4,
                        help='number of simultaneous processes (cores) to use')
    parser.add_argument('-option_parallels', action='store', type=int, default=1,
                        help='Switch to use or not use parallels (1 - use parallels, 0 - do not)')
    parser.add_argument('-debug', action='store', type=int, default=0,
                        help='Boolean for debugging to turn it on or off (default 0 is off')
    return parser

In [3]:
def solar_calcs_NOAA(year, julian_day_of_year, time_frac, longitude_deg, latitude_deg, nsteps):
    """ NOAA calculations to determine the position of the sun and distance to sun

    Sun position based on NOAA solar calculator
    Earth-sun distance based on Stamnes (2015)

    Parameters
    ----------
    year : np.array
        array of the year associated with each time step
    julian_day_of_year : np.array
        julian day of year associated with each time step
    time_frac : np.array
        time (hour + minute / 60) of each time step
    longitude_deg : float
        longitude in degrees
    latitude_deg : float
        latitude in degrees

    Returns
    -------
    SolarZenithAngleCorr_rad : np.array
        Solar zenith angle [radians] corrected for atmospheric refraction
    SolarAzimuthAngle_rad : np.array
        Solar azimuth angle [radians] based on degrees clockwise from north
    rm_r2 : np.array
        Squared mean earth-sun distance normalized by instantaneous earth-sun distance
    """
    julianday_NOAA = np.zeros((nsteps))
    julianCentury = np.zeros((nsteps))
    GeomMeanLongSun_deg = np.zeros((nsteps))
    GeomMeanLongSun_rad = np.zeros((nsteps))
    GeomMeanAnomSun_deg = np.zeros((nsteps))
    GeomMeanAnomSun_rad = np.zeros((nsteps))
    EccentEarthOrbit = np.zeros((nsteps))
    SunEqofCtr = np.zeros((nsteps))
    SunTrueLong_deg = np.zeros((nsteps))
    SunAppLong_deg = np.zeros((nsteps))
    SunAppLong_rad = np.zeros((nsteps))
    MeanObliqEcliptic_deg = np.zeros((nsteps))
    ObliqCorr_deg = np.zeros((nsteps))
    ObliqCorr_rad = np.zeros((nsteps))
    SunDeclin_deg = np.zeros((nsteps))
    SunDeclin_rad = np.zeros((nsteps))
    VarY = np.zeros((nsteps))
    EqofTime = np.zeros((nsteps))
    TrueSolarTime = np.zeros((nsteps))
    HourAngle_deg = np.zeros((nsteps))
    HourAngle_rad = np.zeros((nsteps))
    SolarZenithAngle_deg = np.zeros((nsteps))
    SolarZenithAngle_rad = np.zeros((nsteps))
    SolarElevationAngle_deg = np.zeros((nsteps))
    SolarElevationAngle_rad = np.zeros((nsteps))
    ApproxAtmosRefrac_deg = np.zeros((nsteps))
    SolarElevationAngleCorr_deg = np.zeros((nsteps))
    SolarZenithAngleCorr_deg = np.zeros((nsteps))
    SolarZenithAngleCorr_rad = np.zeros((nsteps))
    SolarAzimuthAngle_deg = np.zeros((nsteps))
    SolarAzimuthAngle_rad = np.zeros((nsteps))
    rm_r2 = np.zeros((nsteps))

    # Julian day
    #  +1 accounts for the fact that day 1 is January 1, 1900
    #  2415018.5 converts from 1900 to NOAA Julian day of year
    julianday_NOAA = (np.floor(365.25*(year-1900)+1) + julian_day_of_year + (time_frac-input.timezone )/24 +
                      2415018.5)
    # Julian Century
    julianCentury = (julianday_NOAA-2451545) / 36525
    # Geom Mean Long Sun
    GeomMeanLongSun_deg = (280.46646 + julianCentury * (36000.76983 + julianCentury*0.0003032)) % 360
    GeomMeanLongSun_rad = GeomMeanLongSun_deg * np.pi/180
    # Geom Mean Anom Sun
    GeomMeanAnomSun_deg = 357.52911 + julianCentury * (35999.05029 - 0.0001537*julianCentury)
    GeomMeanAnomSun_rad = GeomMeanAnomSun_deg * np.pi/180
    # Eccent Earth Orbit
    EccentEarthOrbit = 0.016708634 - julianCentury * (0.000042037 + 0.0000001267*julianCentury)
    # Sun Eq of Ctr
    SunEqofCtr = (np.sin(GeomMeanAnomSun_rad) * (1.914602 - julianCentury * (0.004817 + 0.000014*julianCentury)) +
                  np.sin(2 * GeomMeanAnomSun_rad) * (0.019993 - 0.000101*julianCentury) +
                  np.sin(3 * GeomMeanAnomSun_rad) * 0.000289)
    # Sun True Long
    SunTrueLong_deg = GeomMeanLongSun_deg + SunEqofCtr
    # Sun True Anom
    #SunTrueAnom_deg = GeomMeanAnomSun_deg + SunEqofCtr
    # Sun Rad Vector [AUs]
    #SunRadVector = ((1.000001018 * (1 - EccentEarthOrbit * EccentEarthOrbit)) /
    #                (1 + EccentEarthOrbit * np.cos(SunTrueAnom_rad)))
    # Sun App Long
    SunAppLong_deg = SunTrueLong_deg - 0.00569 - 0.00478 * np.sin((125.04 - 1934.136*julianCentury) * np.pi/180)
    SunAppLong_rad = SunAppLong_deg * np.pi/180
    # Mean Obliq Ecliptic
    MeanObliqEcliptic_deg = (23 + (26 + ((21.448 - julianCentury * (46.815 + julianCentury * (0.00059 -
                             julianCentury * 0.001813)))) / 60) / 60)
    # Obliq Corr
    ObliqCorr_deg = MeanObliqEcliptic_deg + 0.00256 * np.cos((125.04 - 1934.136*julianCentury) * np.pi/180)
    ObliqCorr_rad = ObliqCorr_deg * np.pi/180
    # Sun Rt Ascen
    #SunRtAscen_deg = (180/np.pi * np.arctan((np.cos(ObliqCorr_rad) * np.sin(SunAppLong_rad)) /
    #                                        np.cos(SunAppLong_rad)))
    # Sun Declin
    SunDeclin_deg = 180/np.pi * np.arcsin(np.sin(ObliqCorr_rad) * np.sin(SunAppLong_rad))
    SunDeclin_rad = SunDeclin_deg * np.pi/180
    # VarY
    VarY = np.tan(ObliqCorr_deg / 2 * np.pi/180) * np.tan(ObliqCorr_deg / 2 * np.pi/180)
    # Eq of Time [min]
    EqofTime = (4 * 180/np.pi * (VarY * np.sin(2 * GeomMeanLongSun_rad) - 2 * EccentEarthOrbit *
                np.sin(GeomMeanAnomSun_rad) + 4 * EccentEarthOrbit * VarY * np.sin(GeomMeanAnomSun_rad) *
                np.cos(2 * GeomMeanLongSun_rad) - 0.5 * VarY * VarY * np.sin(4 * GeomMeanLongSun_rad) - 1.25 *
                EccentEarthOrbit * EccentEarthOrbit * np.sin(2 * GeomMeanAnomSun_rad)))
    # True Solar Time [min]
    TrueSolarTime = (time_frac*60*1440 + time_frac*60 + EqofTime + 4*longitude_deg - 60*input.timezone) % 1440
    # Hour Angle
    HourAngle_deg[TrueSolarTime/4 < 0] = TrueSolarTime[TrueSolarTime/4 < 0] / 4 + 180
    HourAngle_deg[TrueSolarTime/4 >= 0] = TrueSolarTime[TrueSolarTime/4 >= 0] / 4 - 180
    HourAngle_rad = HourAngle_deg * np.pi/180
    # Solar Zenith Angle (deg)
    SolarZenithAngle_deg = (180/np.pi * np.arccos(np.sin(latitude_deg * np.pi/180) * np.sin(SunDeclin_rad) +
                            np.cos(latitude_deg * np.pi/180) * np.cos(SunDeclin_rad) * np.cos(HourAngle_rad)))
    SolarZenithAngle_rad = SolarZenithAngle_deg * np.pi/180
    # Solar Elevation Angle (deg)
    SolarElevationAngle_deg = 90 - SolarZenithAngle_deg
    SolarElevationAngle_rad = SolarElevationAngle_deg * np.pi/180
    # Approx Atmospheric Refraction (deg)
    ApproxAtmosRefrac_deg = -20.772 / np.tan(SolarElevationAngle_rad)
    ApproxAtmosRefrac_deg[SolarElevationAngle_deg > 85] = 0
    mask = [(SolarElevationAngle_deg > 5) & (SolarElevationAngle_deg <= 85)]
    ApproxAtmosRefrac_deg[mask] = (
            58.1 / np.tan(SolarElevationAngle_rad[mask]) - 0.07 / ((np.tan(SolarElevationAngle_rad[mask]))**3) +
            0.000086 / ((np.tan(SolarElevationAngle_rad[mask]))**5))
    mask = [(SolarElevationAngle_deg > -0.575) & (SolarElevationAngle_deg <= 5)]
    ApproxAtmosRefrac_deg[mask] = (
            1735 + SolarElevationAngle_deg[mask] * (-518.2 + SolarElevationAngle_deg[mask] *
            (103.4 + SolarElevationAngle_deg[mask] * (-12.79 + SolarElevationAngle_deg[mask]*0.711))))
    ApproxAtmosRefrac_deg = ApproxAtmosRefrac_deg / 3600
    # Solar Elevation Correct for Atm Refraction
    SolarElevationAngleCorr_deg = SolarElevationAngle_deg + ApproxAtmosRefrac_deg
    # Solar Zenith Angle Corrected for Atm Refraction
    SolarZenithAngleCorr_deg = 90 - SolarElevationAngleCorr_deg
    SolarZenithAngleCorr_rad = SolarZenithAngleCorr_deg * np.pi/180
    # Solar Azimuth Angle (deg CW from N)
    SolarAzimuthAngle_deg[HourAngle_deg > 0] = (
            ((180/np.pi * (np.arccos(((np.sin(latitude_deg * np.pi/180) *
              np.cos(SolarZenithAngle_rad[HourAngle_deg > 0])) - np.sin(SunDeclin_rad[HourAngle_deg > 0])) /
              (np.cos(latitude_deg * np.pi/180) * np.sin(SolarZenithAngle_rad[HourAngle_deg > 0])))) + 180) / 360 -
             np.floor((180/np.pi * (np.arccos(((np.sin(latitude_deg * np.pi/180) *
             np.cos(SolarZenithAngle_rad[HourAngle_deg > 0])) - np.sin(SunDeclin_rad[HourAngle_deg > 0])) /
             (np.cos(latitude_deg * np.pi/180) * np.sin(SolarZenithAngle_rad[HourAngle_deg > 0])))) + 180) / 360))
            * 360)
    SolarAzimuthAngle_deg[HourAngle_deg <= 0] = (
            ((540 - 180/np.pi * (np.arccos(((np.sin(latitude_deg * np.pi/180) *
              np.cos(SolarZenithAngle_rad[HourAngle_deg <= 0])) - np.sin(SunDeclin_rad[HourAngle_deg <= 0])) /
              (np.cos(latitude_deg * np.pi/180) * np.sin(SolarZenithAngle_rad[HourAngle_deg <= 0]))))) / 360 -
             np.floor((540 - 180/np.pi * (np.arccos(((np.sin(latitude_deg * np.pi/180) *
             np.cos(SolarZenithAngle_rad[HourAngle_deg <= 0])) - np.sin(SunDeclin_rad[HourAngle_deg <= 0])) /
             (np.cos(latitude_deg * np.pi/180) * np.sin(SolarZenithAngle_rad[HourAngle_deg <= 0]))))) / 360)) * 360)
    SolarAzimuthAngle_rad = SolarAzimuthAngle_deg * np.pi/180
    # Distance from sun based on eccentricity of orbit (r/rm)^2 based on Stamnes (2015)
    # Day number [radians]
    dn_rad = julian_day_of_year * 2 * np.pi / 365
    rm_r2 = (1 / (1.000110 + 0.034221 * np.cos(dn_rad) + 0.001280 * np.sin(dn_rad) + 0.000719 *
                  np.cos(2 * dn_rad) + 0.000077 * np.sin(2 * dn_rad)))**2
    return SolarZenithAngleCorr_rad, SolarAzimuthAngle_rad, rm_r2


def CrankNicholson(Td, Tair, i, debris_thickness, N, h, C, a_Crank, b_Crank, c_Crank, d_Crank, A_Crank, S_Crank):
    """ Run Crank-Nicholson scheme to obtain debris temperature

    Parameters
    ----------
    Td : np.array
        debris temperature [k] (rows = internal layers, columns = timestep)
    Tair : np.array
        air temperature [K]
    i : int
        step number
    debris_thickness : float
        debris thickness [m]
    N : int
        number of layers
    h : float
        height of debris layers [m]
    C : float
        constant defined by Reid and Brock (2010) for Crank-Nicholson Scheme
    a_Crank,

    Returns
    -------
    Td : np.array
        updated debris temperature [k] (rows = internal layers, columns = timestep)
    """
    # Calculate temperature profile in the debris
    # For t = 0, which is i = 1, assume initial condition of linear temperature profile in the debris
    if i == 0:
        Td_gradient = (Td[0,0] - Td[N-1,0])/debris_thickness

        # CODE IMPROVEMENT HERE: TD CALCULATION SKIPPED ONE
        for j in np.arange(1,N-1):
            Td[j,0] = Td[0,0] - (j*h)*Td_gradient

    else:
        # Perform Crank-Nicholson Scheme
        for j in np.arange(1,N-1):
            # Equations A8 in Reid and Brock (2010)
            a_Crank[j,i] = C
            b_Crank[j,i] = 2*C+1
            c_Crank[j,i] = C

            # Equations A9 in Reid and Brock (2010)
            if j == 1:
                d_Crank[j,i] = C*Td[0,i] + C*Td[0,i-1] + (1-2*C)*Td[j,i-1] + C*Td[j+1,i-1]
            elif j < (N-2):
                d_Crank[j,i] = C*Td[j-1,i-1] + (1-2*C)*Td[j,i-1] + C*Td[j+1,i-1]
            elif j == (N-2):
                d_Crank[j,i] = 2*C*Td[N-1,i] + C*Td[N-3,i-1] + (1-2*C)*Td[N-2,i-1]
            # note notation:
            #  "i-1" refers to the past
            #  "j-1" refers to the cell above it
            #  "j+1" refers to the cell below it


            # Equations A10 and A11 in Reid and Brock (2010)
            if j == 1:
                A_Crank[j,i] = b_Crank[j,i]
                S_Crank[j,i] = d_Crank[j,i]
            else:
                A_Crank[j,i] = b_Crank[j,i] - a_Crank[j,i] / A_Crank[j-1,i] * c_Crank[j-1,i]
                S_Crank[j,i] = d_Crank[j,i] + a_Crank[j,i] / A_Crank[j-1,i] * S_Crank[j-1,i]

        # Equations A12 in Reid and Brock (2010)
        for j in np.arange(N-2,0,-1):
            if j == (N-2):
                Td[j,i] = S_Crank[j,i] / A_Crank[j,i]
            else:
                Td[j,i] = 1 / A_Crank[j,i] * (S_Crank[j,i] + c_Crank[j,i] * Td[j+1,i])
    return Td


def calc_surface_fluxes(Td_i, Tair_i, RH_AWS_i, u_AWS_i, Sin_i, Lin_AWS_i, Rain_AWS_i, snow_i, P, Albedo, k,
                        a_neutral_debris, h, dsnow_t0, tsnow_t0, snow_tau_t0, ill_angle_rad_i, a_neutral_snow,
                        debris_thickness,
                        option_snow=0, option_snow_fromAWS=0):
    """ Calculate surface energy fluxes for timestep i

    Snow model uses a modified version of Tarboten and Luce (1996) to compute fluxes
      - Sin calculated above, not with their local slope and illumination corrections though
        they are likely similar
      - Ground heat flux is computed from the debris, not using their estimates from diurnal
        soil temperatures
      - Do not use their temperature threshold for snowfall, but use set value
      - For P_flux, we do not include the snow fall component because it alters the cold content of the snow pack
        therefore we don't want to double count this energy
      - Do not account for wind redistribution of snow, which they state is site specific
      - Do not iterate to solve snow temperature, but do a depth average
      - Solve for the thermal conductivity at the debris/ice interface using the depth of snow and debris height

    Limitation:
      - If allow all negative energy to warm up the snowpack and the snowpack is very thin (< 1 cm), then the
        change in temperature can be extreme (-10 to -1000s of degrees), which is unrealistic.
        More realistic is that the snowpack may change its temperature and the remaining energy will be
        transferred to also cool the debris layer. Set maximum temperature change of the snow pack during any given
        time step to 1 degC.

    Note: since partitioning rain into snow, units are automatically m w.e.
          hence, the density of snow is not important

    Future work:
      - Currently, the debris/ice interface is set to 273.15.
        This is fine during the ablation season; however, when the debris freezes in the winter
      - Snow melt water should theoretically percolate into the debris and transfer energy

    Parameters
    ----------
    Td_i : np.array
        debris temperature
    Tair_i, RH_AWS_i, u_AWS_i, Sin_i, Lin_AWS_i, Rain_AWS_i, snow_i : floats
        meteorological data
    P : float
        pressure [Pa]
    Albedo, k, a_neutral_debris : floats
        debris albedo, thermal conductivity, and turbulent heat flux transfer coefficient (from surface roughness)
    h : float
        debris layer height [m]
    dsnow_t0, tsnow_t0, snow_tau_t0
        snow depth, temperature and dimensionless age at start of time step before any snow or melt has occurred
    ill_angle_rad_i : float
        solar illumination angle used to adjust snow albedo
    a_neutral_snow : float
        snow turbulent heat flux transfer coefficient (based on surface roughness)
    option_snow : int
        switch to use snow model (1) or not (0)
    option_snow_fromAWS : int
        switch to use snow depth (1) instead of snow fall (0)

    Returns
    -------
    F_Ts_i, Rn_i, LE_i, H_i, P_flux_i, Qc_i : floats
        Energy fluxes [W m-2]
    dF_Ts_i, dRn_i, dLE_i, dH_i, dP_flux_i, dQc_i : floats
        Derivatives of energy fluxes
    dsnow_i : float
        Snow depth [mwe] at end of time step
    tsnow_i : float
        Snow temperature at end of time step
    snow_tau_i : float
        Non-dimensional snow age at end of time step
    """
    # Snow depth [m w.e.]
    dsnow_i = dsnow_t0 + snow_i
    snow_tau_i = snow_tau_t0
    tsnow_i = 0

    # First option: Snow depth is based on snow fall, so need to melt snow
    if dsnow_i > 0 and option_snow==1 and option_snow_fromAWS == 0:
        tsnow_i = (dsnow_t0 * tsnow_t0 + snow_i * Tair_i) / dsnow_i

        # Thermal conductivity at debris/snow interface assuming conductance resistance is additive
        #  estimating the heat transfer through dsnow_eff layer of snow and h_eff layer of debris
        # Tarboten and Luce (1996) use effective soil depth of 0.4 m for computing the ground heat transfer
        if debris_thickness < 0.4:
            h_eff = debris_thickness
        else:
            h_eff = 0.4
        if dsnow_i < 0.4:
            dsnow_eff = dsnow_i
        else:
            dsnow_eff = 0.4
        k_snow_interface = (h_eff + dsnow_eff) / (dsnow_eff/input.k_snow + h_eff/k)
        # Previously estimating it based on equal parts
        #k_snow_interface = h / ((0.5 * h) / input.k_snow + (0.5*h) / k)

        # Density of air (dry) based on pressure (elevation) and temperature
        #  used in snow calculations, which has different parameterization of turbulent fluxes
        #  compared to the debris
        density_air = P / (287.058 * Tair_i)

        # Albedo
        # parameters representing grain growth due to vapor diffusion (r1), additional effect near
        #  and at freezing point due to melt and refreeze (r2), and the effect of dirt and soot (r3)
        snow_r1 = np.exp(5000 * (1 / 273.16 - 1 / tsnow_i))
        snow_r2 = np.min([snow_r1**10, 1])
        snow_r3 = 0.03 # change to 0.01 if in Antarctica
        # change in non-dimensional snow surface age
        snow_tau_i += (snow_r1 + snow_r2 + snow_r3) / input.snow_tau_0 * input.delta_t
        # new snow affect on snow age
        if snow_i > 0.01:
            snow_tau_i = 0
        elif snow_i > 0:
            snow_tau_i = snow_tau_i * (1 - 100 * snow_i)
        # snow age
        snow_age = snow_tau_i / (1 + snow_tau_i)
        # albedo as a function of snow age and band
        albedo_vd = (1 - input.snow_c_v * snow_age) * input.albedo_vo
        albedo_ird = (1 - input.snow_c_ir * snow_age) * input.albedo_iro
        # increase in albedo based on illumination angle
        #  illumination angle measured relative to the surface normal
        if np.cos(ill_angle_rad_i) < 0.5:
            b_ill = 2
            f_psi = 1/b_ill * ((1 + b_ill) / (1 + 2 * b_ill * np.cos(ill_angle_rad_i)) - 1)
        else:
            f_psi = 0
        albedo_v = albedo_vd + 0.4 * f_psi * (1 - albedo_vd)
        albedo_ir = albedo_ird + 0.4 * f_psi * (1 - albedo_ird)
        albedo_snow = np.mean([albedo_v, albedo_ir])
        # Adjustments to albedo
        # ensure albedo is within bounds
        if albedo_snow > 1:
            albedo_snow = 1
        elif albedo_snow < 0:
            albedo_snow = 0
        # if snow less than 0.1 m, then underlying debris influences albedo
        if dsnow_i < 0.1:
            r_adj = (1 - dsnow_i/0.1)*np.exp(dsnow_i / (2*0.1))
            albedo_snow = r_adj * Albedo + (1 - r_adj) * albedo_snow

        # Snow Energy Balance
        Rn_snow = (Sin_i * (1 - albedo_snow) + input.emissivity_snow * (Lin_AWS_i -
                   (input.stefan_boltzmann * tsnow_i**4)))
        H_snow = a_neutral_snow * density_air * input.cA * u_AWS_i * (Tair_i - tsnow_i)
        # Vapor pressure above snow assumed to be saturated
        # Vapor pressure (e, Pa) computed using Clasius-Clapeyron Equation and Relative Humidity
        #  611 is the vapor pressure of ice and liquid water at melting temperature (273.15 K)
        eZ_Saturated = 611 * np.exp(input.Lv / input.R_const * (1 / 273.15 - 1 / Tair_i))
        eZ = RH_AWS_i * eZ_Saturated
        # Vapor pressure of snow based on temperature (Colbeck, 1990)
        e_snow = input.eS_snow * np.exp(2838 * (tsnow_i - 273.15) / (0.4619 * tsnow_i * 273.15))
        if e_snow > input.eS_snow:
            e_snow = input.eS_snow
        LE_snow = 0.622 * input.Ls / (input.Rd * Tair_i) * a_neutral_snow * u_AWS_i * (eZ - e_snow)
        Pflux_snow = (Rain_AWS_i * (input.Lf * input.density_water + input.cW * input.density_water *
                                    (np.max([273.15, Tair_i]) - 273.15)) / input.delta_t)
        Qc_snow_debris = k_snow_interface * (Td_i[0] - tsnow_i)/h

        # Net energy available for snow depends on latent heat flux
        # if Positive LE: Air > snow vapor pressure (condensation/resublimation)
        #  energy released and available to melt the snow (include LE in net energy)
        if LE_snow > 0:
            Fnet_snow = Rn_snow + H_snow + LE_snow + Pflux_snow + Qc_snow_debris
            snow_sublimation = 0
        # if Negative LE: Air < snow vapor pressure (sublimation/evaporation)
        #  energy consumed and snow sublimates (do not include LE in net energy)
        else:
            Fnet_snow = Rn_snow + H_snow + Pflux_snow + Qc_snow_debris
            # Snow sublimation [m w.e.]
            snow_sublimation = -1 * LE_snow / (input.density_water * input.Lv) * input.delta_t

        # Cold content of snow [W m2]
        Qcc_snow = input.cSnow * input.density_water * dsnow_i * (273.15 - tsnow_i) / input.delta_t

        # Max energy spent cooling snowpack based on 1 degree temperature change
        Qcc_snow_neg1 = -1 * input.cSnow * input.density_water * dsnow_i / input.delta_t

        # If Fnet_snow is positive and greater than cold content, then energy is going to warm the
        # snowpack to melting point and begin melting the snow.
        if Fnet_snow > Qcc_snow:
            # Snow warmed up to melting temperature
            tsnow_i = 273.15
            Fnet_snow -= Qcc_snow
            Fnet_snow2debris = 0
        elif Fnet_snow < Qcc_snow_neg1:
            # Otherwise only changes the temperature in the snowpack and the debris
            # limit the change in snow temperature
            tsnow_i -= 1
            Fnet_snow2debris = Fnet_snow - Qcc_snow_neg1
            Fnet_snow = 0
        else:
            # Otherwise only changes the temperature
            tsnow_i += Fnet_snow / (input.cSnow * input.density_water * dsnow_i) * input.delta_t
            Fnet_snow = 0
            Fnet_snow2debris = 0

        # Snow melt [m snow] with remaining energy, if any
        snow_melt_energy = Fnet_snow / (input.density_water * input.Lf) * input.delta_t

        # Total snow melt
        snow_melt = snow_melt_energy + snow_sublimation

        # Snow depth [m w.e.]
        dsnow_i -= snow_melt
        if dsnow_i < 0:
            dsnow_i = 0
        if dsnow_i == 0:
            snow_tau_i = 0

        # Solve for temperature in debris
        #  Rn, LE, H, and P equal 0
        Rn_i = 0
        LE_i = 0
        H_i = 0
        Qc_i = k * (Td_i[1] - Td_i[0]) / h
        P_flux_i = 0
        Qc_snow_i = -Qc_snow_debris
        F_Ts_i = Rn_i + LE_i + H_i + Qc_i + P_flux_i + Qc_snow_i  + Fnet_snow2debris

        dRn_i = 0
        dLE_i = 0
        dH_i = 0
        dQc_i = -k/h
        dP_flux_i = 0
        dQc_snow_i = -k_snow_interface/h
        dF_Ts_i = dRn_i + dLE_i + dH_i + dQc_i + dP_flux_i + dQc_snow_i

    # Second option: Snow depth is prescribed from AWS, so don't need to melt snow
    elif dsnow_i > 0 and option_snow==1 and option_snow_fromAWS == 1:
        dsnow_i = snow_i
        tsnow_i = Tair_i
        if tsnow_i > 273.15:
            tsnow_i = 273.15

        # Thermal conductivity at debris/snow interface assuming conductance resistance is additive
        #  estimating the heat transfer through dsnow_eff layer of snow and h_eff layer of debris
        # Tarboten and Luce (1996) use effective soil depth of 0.4 m for computing the ground heat transfer
        if debris_thickness < 0.4:
            h_eff = debris_thickness
        else:
            h_eff = 0.4
        if dsnow_i < 0.4:
            dsnow_eff = dsnow_i
        else:
            dsnow_eff = 0.4
        k_snow_interface = (h_eff + dsnow_eff) / (dsnow_eff/input.k_snow + h_eff/k)

        Qc_snow_debris = k_snow_interface * (Td_i[0] - tsnow_i)/h

        # Solve for temperature in debris
        #  Rn, LE, H, and P equal 0
        Rn_i = 0
        LE_i = 0
        H_i = 0
        Qc_i = k * (Td_i[1] - Td_i[0]) / h
        P_flux_i = 0
        Qc_snow_i = -Qc_snow_debris
        Fnet_snow2debris = 0
        F_Ts_i = Rn_i + LE_i + H_i + Qc_i + P_flux_i + Qc_snow_i

        dRn_i = 0
        dLE_i = 0
        dH_i = 0
        dQc_i = -k/h
        dP_flux_i = 0
        dQc_snow_i = -k_snow_interface/h
        dF_Ts_i = dRn_i + dLE_i + dH_i + dQc_i + dP_flux_i + dQc_snow_i

    else:
        # Debris-covered glacier Energy Balance (no snow)
        if Rain_AWS_i > 0:
            # Vapor pressure (e, Pa) computed using Clasius-Clapeyron Equation and Relative Humidity
            #  611 is the vapor pressure of ice and liquid water at melting temperature (273.15 K)
            # if raining, assume the surface is saturated
            eS_Saturated = 611 * np.exp(-input.Lv / input.R_const * (1 / Td_i[0] - 1 / 273.15))
            eS = eS_Saturated
            eZ_Saturated = 611 * np.exp(-input.Lv / input.R_const * (1 / Tair_i - 1 / 273.15))
            eZ = RH_AWS_i * eZ_Saturated
            LE_i = (0.622 * input.density_air_0 / input.P0 * input.Lv * a_neutral_debris * u_AWS_i
                    * (eZ -eS))
        else:
            LE_i = 0
        Rn_i = Sin_i * (1 - Albedo) + input.emissivity * (Lin_AWS_i - (5.67e-8 * Td_i[0]**4))
        H_i = (input.density_air_0 * (P / input.P0) * input.cA * a_neutral_debris * u_AWS_i *
               (Tair_i - Td_i[0]))
        P_flux_i = input.density_water * input.cW * Rain_AWS_i / input.delta_t * (Tair_i - Td_i[0])
        Qc_i = k * (Td_i[1] - Td_i[0]) / h
        F_Ts_i = Rn_i + LE_i + H_i + Qc_i + P_flux_i

        # Derivatives
        if Rain_AWS_i > 0:
            dLE_i = (-0.622 * input.density_air_0 / input.P0 * input.Lv * a_neutral_debris *
                     u_AWS_i * 611 * np.exp(-input.Lv / input.R_const * (1 / Td_i[0] - 1 / 273.15))
                     * (input.Lv / input.R_const * Td_i[0]**-2))
        else:
            dLE_i = 0
        dRn_i = -4 * input.emissivity * 5.67e-8 * Td_i[0]**3
        dH_i = -1 * input.density_air_0 * P / input.P0 * input.cA * a_neutral_debris * u_AWS_i
        dP_flux_i = -input.density_water * input.cW * Rain_AWS_i/ input.delta_t
        dQc_i = -k / h
        dF_Ts_i = dRn_i + dLE_i + dH_i + dQc_i + dP_flux_i

    return (F_Ts_i, Rn_i, LE_i, H_i, P_flux_i, Qc_i, dF_Ts_i, dRn_i, dLE_i, dH_i, dP_flux_i, dQc_i,
            dsnow_i, tsnow_i, snow_tau_i)

In [6]:
def main(list_packed_vars):
    """
    Run melt model for list of latlons

    Parameters
    ----------
    list_packed_vars : list
        list of packed variables that enable the use of parallels

    Returns
    -------
    ...
    """
    # Unpack variables
    count = list_packed_vars[0]
    latlon_list = list_packed_vars[1]

In [7]:
time_start = time.time()
# parser = getparser()
# args = parser.parse_args()

# if args.debug == 1:
#     debug = True
# else:
#     debug = False

#=====
print('\nREPLACE ME WITH ARG PARSER\n')
class batman():
    pass
args = batman()
args.option_ordered=1
args.option_parallels=0
args.num_simultaneous_processes=3
#=====

time_start = time.time()

# Date of start of simulation
date_start = datetime.datetime.today().strftime('%Y%m%d')

# Number of cores for parallel processing
if args.option_parallels != 0:
    num_cores = int(np.min([len(input.latlon_list), args.num_simultaneous_processes]))
else:
    num_cores = 1

# Latlon batches to pass for parallel processing
latlon_list_batches = split_list(input.latlon_list, n=num_cores, option_ordered=args.option_ordered)

# Pack variables for multiprocessing
list_packed_vars = []
for count, latlon_list_batch in enumerate(latlon_list_batches):
    list_packed_vars.append([count, latlon_list_batch])

# Parallel processing
if args.option_parallels != 0:
    print('Processing in parallel with ' + str(args.num_simultaneous_processes) + ' cores...')
    with multiprocessing.Pool(args.num_simultaneous_processes) as p:
        p.map(main,list_packed_vars)
# If not in parallel, then only should be one loop
else:
    # Loop through the chunks and export bias adjustments
    for n in range(len(list_packed_vars)):
        main(list_packed_vars[n])


REPLACE ME WITH ARG PARSER



In [5]:
for nlatlon, latlon in enumerate(input.latlon_list):
    print(nlatlon, latlon)
    
#     print('\n\nSHOULD TRY TO SET THIS UP BETTER AS AN OBJECT-ORIENTED PROGRAM\n\n')
    
    # ===== Meteorological data - GET ALL DATA FROM THE CLIMATE DATA OR ASSUMPTIONS =====
    met_data = np.genfromtxt(input.met_data_fullfn, delimiter=',', skip_header=1)
    met_data = met_data[input.start_idx:input.end_idx,:]
    long_deg = input.lon_AWS_deg
    lat_deg = input.lat_AWS_deg
    Slope_AWS_rad = input.slope_AWS_deg*(np.pi/180)
    Aspect_AWS_rad = input.aspect_AWS_deg*(np.pi/180)
    P_AWS = input.P0*np.exp(-0.0289644*9.81*input.Elev_AWS/(8.31447*288.15))  # Pressure at Pyramid Station

    # Select data
    year = met_data[:,0].astype(int)
    month = met_data[:,1].astype(int)
    day = met_data[:,2].astype(int)
    hour = met_data[:,3].astype(int)
    minute = met_data[:,4].astype(int)
    Tair_AWS = met_data[:,5] + 273.15 #celsius to kelvin
    RH_AWS = met_data[:,6] / 100 # percentage to decimal
    u_AWS_raw = met_data[:,7]
    Rain_AWS = met_data[:,14]/1000 # mm to meters
    Sin_AWS = met_data[:,10]
    Sout_AWS = met_data[:,11]
    Lin_AWS = met_data[:,12]
    Snow_AWS = met_data[:,15]

    # Albedo from AWS
    albedo_AWS = np.zeros(Sin_AWS.shape)
    albedo_AWS[Sin_AWS > 0] = Sout_AWS[Sin_AWS > 0] / Sin_AWS[Sin_AWS > 0]
    albedo_AWS[albedo_AWS > 1] = 1


0 (28.0, 86.75)
